In [ ]:
%%capture
%pip install accelerate bitsandbytes -q
!pip install -Uq transformers

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# init setup


In [ ]:
import time

# Get the current time struct
current_time = time.localtime()

# Extract date and short month name
date = time.strftime("%d", current_time)
short_month = time.strftime("%b", current_time)

print(date)
print(short_month)

31
Jul


In [ ]:
from pathlib import Path
path = Path('/content/drive/Shareddrives/sentence_genration')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

base_path='/content/drive/MyDrive/Sentiment_Analysis_Roberta/'
base_intent_path='/content/drive/MyDrive/Intent_Analysis/'

Mounted at /content/drive


In [ ]:
import sys
# adding utils to the system path
sys.path.insert(0, base_path+'SA_utils')
sys.path.insert(0, base_intent_path+'intent_utils')


In [ ]:
from main import *
from config import current_possible_intents as intents, MOBILE_MODEL_PTH, MOBILE_TOKENIZER_PTH, Mobile_competitor_dict_pth, Mobile_intent_dict_pth_and_sheet_info
from intent_utils import *
from pro_con_sa import *
from subdomain_analysis_funcs import *
from subdomain_analysis_funcs import *

Loading files...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Files loaded.


# Loading data

In [ ]:
import time
from google.colab import drive
from pathlib import Path
import sys
from main import *
from config import current_possible_intents as intents, MOBILE_MODEL_PTH, MOBILE_TOKENIZER_PTH, Mobile_competitor_dict_pth, Mobile_intent_dict_pth_and_sheet_info
from intent_utils import *
from pro_con_sa import *
from subdomain_analysis_funcs import *
from subdomain_analysis_funcs import *
import torch
import pandas as pd
from subdomain_analysis_funcs import subdomain_splitter
import IPython
from transformers import pipeline, AutoTokenizer, BitsAndBytesconfig, AutoModelForCausalLM
from tqdm import tqdm
import pickle
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from numba import cuda
import gc

In [ ]:
# items_to_do_direct_opp=[
#     '^(?:(?!saved?|back ?-?up|\\beditor\\b|\\beras[eding]*\\b|\\bedit[ingedabl]*|google|\\bplayback|\\bscreen|\\bdisplay|drive\\b|bright[erst]*\\b|\\bu?n?responsive\\b).)*(\\bimages?\\b|\\bpictures?\\b|\\bpics?\\b|\\bportraits?\\b|\\bphotos?\\b)(?!.*back ?-?up|(?!.* without).*\\beditor\\b|erase\\b|(?!.* without).*\\bedit[edingabl]*\\b|.*\\bplayback\\b|.*\\bscreen|.*\\bdisplay|.*drive\\b|.*bright\\b|(?!.* without).*\\bsettings?\\b|.*\\bu?n?responsive\\b|saved?)',
#     ]

import pickle
if 'Subdom_re_dict' in globals() :
    pass

else :
    file_path = '/content/drive/Shareddrives/sentence_genration/misc/Subdom_re_dict_29_Jul.pkl'
    with open(file_path, 'rb') as f:
        Subdom_re_dict = pickle.load(f)

In [ ]:
for i in Subdom_re_dict:
    temp_df=pd.DataFrame(Subdom_re_dict[i])
    # print(temp_df.info())
    # break
    print(i)
    print(temp_df['Camera'].value_counts())
    print()

^(?:(?!saved?|back ?-?up|\beditor\b|\beras[eding]*\b|\bedit[ingedabl]*|google|\bplayback|\bscreen|\bdisplay|drive\b|bright[erst]*\b|\bu?n?responsive\b).)*(\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b)(?!.*back ?-?up|(?!.* without).*\beditor\b|erase\b|(?!.* without).*\bedit[edingabl]*\b|.*\bplayback\b|.*\bscreen|.*\bdisplay|.*drive\b|.*bright\b|(?!.* without).*\bsettings?\b|.*\bu?n?responsive\b|saved?)
Camera
 1.0    1061
-1.0      68
Name: count, dtype: int64

^(?:(?!streami?n?g?|watchi?n?g?|back ?-?up|\beditor\b|\beras[edings]*\b|\bedit[ingedabl]*\b|\bplayback|\bscreen|\bdisplay|drive\b|\bplay|\bwatch|\bu?n?responsive\b).)*(?:\bvideos?\b|\bvedios?\b)(?! back ?-?up|(?!.* without).*\beditor\b|.*\beras[edings]*\b|(?!.* without).*\bedit[edingabl]*\b|.*\bplayback\b|.*\bscreen|.*\bdisplay|.*drive\b| calls?i?n?g?s?\b|.*\bplay|.*settings?|.*\bu?n?responsive\b|.*streami?n?g?|.*watchi?n?g?|\s*appointment)
Camera
 1.0    222
-1.0     22
Name: count, dtype: int64

video quality


In [ ]:
len(Subdom_re_dict)

37

# Final Funcs

## Model Loading

In [ ]:
def load_llama3(model_id = 'Orenguteng/Llama-3-8B-Lexi-Uncensored', model_path = '/content/drive/Shareddrives/sentence_genration/llama3/untrained/model'):


  compute_dtype = getattr(torch, "float16")
  bnb_config = BitsAndBytesconfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=False,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=compute_dtype,
  )

  model = AutoModelForCausalLM.from_pretrained(
      model_path,
      device_map="auto",
      quantization_config=bnb_config,
  )

  tokenizer = AutoTokenizer.from_pretrained(
      model_id,
      # token=access_token,
      cache_dir='/content/drive/Shareddrives/sentence_genration/mistral/Mistral-7B-v0.3/Tokenizer',

      padding_side='left',
      quantization_config=bnb_config,

      )
  tokenizer.pad_token = tokenizer.eos_token


  return model, tokenizer
def load_mistral(model_id = 'mistralai/Mistral-7B-Instruct-v0.3',
                 model_path = '/content/drive/Shareddrives/sentence_genration/mistral/Mistral-7B-Instruct-v0.3/SavedModel',
                 access_token = None #(Optional) Put your access token here incase not downloaded or cache dir is empty
                 ):


  compute_dtype = getattr(torch, "float16")
  bnb_config = BitsAndBytesconfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=False,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=compute_dtype,
  )
  if not access_token:
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=bnb_config,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        # token=access_token,
        cache_dir='/content/drive/Shareddrives/sentence_genration/mistral/Mistral-7B-Instruct-v0.3/Tokenizer',

        padding_side='left',
        quantization_config=bnb_config,

        )


  else:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        cache_dir='/content/drive/Shareddrives/sentence_genration/mistral/Mistral-7B-v0.3/Tokenizer',
        token=access_token,
        device_map="auto",
        quantization_config=bnb_config,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        token=access_token,
        cache_dir='/content/drive/Shareddrives/sentence_genration/mistral/Mistral-7B-v0.3/Tokenizer',

        padding_side='left',
        quantization_config=bnb_config,

        )

  tokenizer.pad_token = tokenizer.eos_token
  return model, tokenizer

# Paraphrase Sentence (Llama3)

In [ ]:
class ParaphraseDataset(Dataset):
    def __init__(self, sentences, n_sentences_to_generate, pos_neg='positive'):
        self.sentences = sentences
        self.n_sentences_to_generate = n_sentences_to_generate
        self.pos_neg=pos_neg
        self.model, self.tokenizer = load_llama3()

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        actually_required_sequences=(self.n_sentences_to_generate // len(self.sentences))
        num_return_sequences =  actually_required_sequences + 30 # At least + 30 the required sentences
        paraphrases = self.generate_paraphrases(sentence, num_return_sequences)

        # Remove duplicates
        paraphrases = list(set(paraphrases))
        if len(paraphrases) < actually_required_sequences:
            paraphrases += self.generate_paraphrases(sentence, 2*(actually_required_sequences - len(paraphrases)))
        paraphrases = paraphrases[:actually_required_sequences]
        return paraphrases


    def generate_paraphrases(self, sentence, num_return_sequences):
        num_beams = num_return_sequences+20  # Ensure num_beams is at least equal to num_return_sequences or greater

        prompt = f"You are a chatbot that returns a single {self.pos_neg} sentiment synonymous or similar sentence using the reference sentences I give you, NOTE: only generate a single sentence not a sentence for each reference sentences. Just the result and nothing else. You always stick to topic of sentences. Here are the sentences:\n"
        # sentence = "Camera is very high quality."

        # Prepare input in the required format
        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": sentence},
        ]

        # Apply chat template and tokenize
        inputs = self.tokenizer.apply_chat_template(messages, add_special_tokens=True, return_tensors='pt', padding=True)
        paraphrase_batch = self.model.generate(inputs, max_new_tokens=50, num_beams=num_beams, num_return_sequences=num_return_sequences, do_sample=True, temperature=0.7)

        paraphrases=[]
        # Decode the generated output
        for output in paraphrase_batch:
            generated_text = self.tokenizer.decode(output, skip_special_tokens=True)
            paraphrases.append(generated_text.split('\n')[-1])
        return paraphrases

def generate_paraphrased_sentences(reference_sentences, n=10, b_size=1, pos_neg='positive'):
    dataset = ParaphraseDataset(reference_sentences, n, pos_neg)
    dataloader = DataLoader(dataset, batch_size=b_size, shuffle=False)

    generated_sentences = set()  # Using a set to discard duplicates
    total_generated = 0

    for batch in dataloader:
        for paraphrases in batch:
            for sentence in paraphrases:
                generated_sentences.add(sentence)
                total_generated += 1

    del dataset
    del dataloader
    torch.cuda.empty_cache()
    gc.collect()
    # device = cuda.get_current_device()
    # device.reset()
    return list(generated_sentences)  # Convert set back to list and take up to n sentences


In [ ]:

# # Example usage
# reference_sentences = [
#     'Camera quality is great.',
#     'Resolution of images is amazing.',
#     'Shutter speed is great.',
#     'I like the images taken from it.',
#     'The color reproduction is excellent.',
#     'Camera is impressive.',
#     'The pictures look sleek and modern.',
#     'The camera app is user-friendly.',
#     'Low light performance is superb.',
#     'The autofocus is very fast.'
# ]

# n = 500  # Number of sentences to generate
# pos_neg='positive'
# positive_sentences = generate_paraphrased_sentences(reference_sentences, n, 1, pos_neg)
# for sentence in positive_sentences:
#     print(sentence)
# print(len(list(set(positive_sentences))))

# for i, j in enumerate(list(set(positive_sentences))):
#     print(str(i)+' '+j)

# print(len(list(set(positive_sentences))))


## Opposite Sentence (Llama3)

In [ ]:

class OppositeSentenceDataset(Dataset):
    def __init__(self, sentences, tokenizer):
        self.sentences = sentences
        self.prompt = "You are a chatbot that returns the opposite of the sentence I give you. Just the result and nothing else. You always stick to what the sentence is originally about."
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        message = [
            {"role": "system", "content": self.prompt},
            {"role": "user", "content": sentence},
        ]
        input_text = self.tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)
        return input_text, sentence

def generate_opposites(model, tokenizer, sentences, batch_size=8):
    dataset = OppositeSentenceDataset(sentences, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    results = []
    for batch in tqdm(dataloader, desc="Generating opposite sentences"):
        inputs, original_sentences = batch
        inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=False).to('cuda')
        outputs = model.generate(**inputs, max_new_tokens=150)
        opposites = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        results.extend(opposites)

    del model
    del tokenizer
    del dataset
    del dataloader
    del inputs
    del outputs

    torch.cuda.empty_cache()
    gc.collect()

    return results

def generate_opposite_sentences(sentences, batch_size=42):
    model, tokenizer = load_llama3()
    # global model
    # global tokenizer

    opposite_sentences = generate_opposites(model, tokenizer, sentences, batch_size=batch_size)

    opposite_sentences = [sentence.split(sentences[i].split(' ')[-1]+"assistant")[-1].strip() for i , sentence in enumerate(opposite_sentences)]

    #Cleanup
    # del model
    # del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    # device = cuda.get_current_device()
    # device.reset()
    return opposite_sentences


In [ ]:
# # Example usage
# reference_sentences = [
#     'Camera quality is great.',
#     'Resolution of images is amazing.',
#     'Shutter speed is great.',
#     'I like the images taken from it.',
#     'The color reproduction is excellent.',
#     'Camera is impressive.',
#     'The pictures look sleek and modern.',
#     'The camera app is user-friendly.',
#     'Low light performance is superb.',
#     'The autofocus is very fast.'
# ]
# opposite_sentences = generate_opposite_sentences(reference_sentences)
# for i, sentence in enumerate(opposite_sentences):
#     print(f'{i}. '+sentence)

## Sentence Generation given reference (Mistral)

In [ ]:
# from google.colab import userdata
# access_token=userdata.get('Hugging_Face_access_token')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Initialize the sentiment analyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
def is_positive(sentence):
    sentiment_scores = sid.polarity_scores(sentence)
    return sentiment_scores['compound'] > 0.05

def is_negative(sentence):
    sentiment_scores = sid.polarity_scores(sentence)
    return sentiment_scores['compound'] < -0.05

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def generate_similar_sentences(reference_sentences, topic, num_sentences=50, pos_neg='positive'):
    model, tokenizer = load_mistral()
    if type(reference_sentences)!= type(['']):
      reference_sentences=[reference_sentences]
    stopping_counter = 0
    gen_sentences=[]
    generated_sent=len(gen_sentences)
    while generated_sent <= num_sentences:


      reference_sents= reference_sentences + gen_sentences

      temp='\n'.join(reference_sents)

      prompt=f"""Give me exactly {num_sentences-generated_sent} different sentences in context of these given reference sentences,
      NOTE:
      1. The sentences should be overall {pos_neg} in sentiment about the context and not just sentiment-less sentences.
      2. Stick to the topic: {topic} and no redundant words like 'ability to'
      3. Paraphrasing is not allowed, make sure each generated sentence is slightly different from each other and talks about slightly different aspect taken from reference sentences
      4. the count of sentences you generate must be exactly {num_sentences-generated_sent}, no less!
      here are the sentences generate exactly {num_sentences-generated_sent} more according to instructions :\n\n {temp}
      """

      messages = [
      {
          "role": "user",
      "content": prompt},

      ]

      inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

      outputs = model.generate(inputs,
                              max_new_tokens=8000,
                              temperature=0.1,
                              top_p=0.99,
                              do_sample = True,
                              pad_token_id=tokenizer.eos_token_id)

      res=tokenizer.decode(outputs[0], skip_special_tokens=True)



      res_sentences=re.split(r'\d+\.', res[len(prompt):])

      for i in res_sentences:
          if i.strip()=='':
              continue
          if pos_neg.lower() == 'positive' and is_positive(i.strip()):
              gen_sentences.append(i.strip())
          elif pos_neg.lower() == 'negative' and is_negative(i.strip()):
              gen_sentences.append(i.strip())
          # print(i.strip())
      generated_sent = len(gen_sentences)
      stopping_counter += 1
      if stopping_counter == 5:
        break
    del model
    del tokenizer
    del outputs
    del inputs
    torch.cuda.empty_cache()
    gc.collect()
    return gen_sentences

In [ ]:
# sentences = [
#     'Camera quality is great.',
#     'Resolution of images is amazing.',
#     'Shutter speed is great.',
#     'I like the images taken from it.',
#     'The color reproduction is excellent.',
#     'Camera is impressive.',
#     'The pictures look sleek and modern.',
#     'The camera app is user-friendly.',
#     'Low light performance is superb.',
#     'The autofocus is very fast.'
# ]

# res = generate_similar_sentences(sentences, 'camera', num_sentences=50, pos_neg='positive')
# for i in res:
#     print(i)

In [ ]:
# len(res)

# Main Code

In [ ]:
Subdom_re_dict2={}
i=0
for j in Subdom_re_dict:
    i+=1
    if i<=30:
      continue
    Subdom_re_dict2[j]=Subdom_re_dict[j]
    if i==37:
        break
# Subdom_re_dict2

In [ ]:
#Main code for Sentence generation on condition
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="Unused kwargs: .*['quant_method']. These kwargs are not used in .*")
warnings.filterwarnings("ignore", message="The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior.")
warnings.filterwarnings("ignore", message="Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.")


id = 0
min_sentence_per_regex= 10
result={}
intent='Camera'

# for i in Subdom_re_dict:
for i in tqdm(Subdom_re_dict2, desc="Ongoing generation"):
    """
    temp_df example:

    Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   UID     129 non-null    int64
 1   Split   129 non-null    object
 2   Camera  129 non-null    float64
    """

    temp_df=pd.DataFrame(Subdom_re_dict2[i])
    print(f'Ongoing {i}')
    print(temp_df[intent].value_counts())

    #Getting val counts
    val_counts=temp_df[intent].value_counts().to_dict()

    #Getting +1 and -1 value counts
    if 1 in val_counts:
        val_count_for_plus_one=val_counts[1]
    else:
        val_count_for_plus_one=0

    if -1 in val_counts:
        val_count_for_minus_one=val_counts[-1]
    else:
        val_count_for_minus_one=0

    #Case 1: when value count are greater than 500 hence we do opposite sentence conversion directly
    if max(val_count_for_plus_one, val_count_for_minus_one)>500:
        print('Case 1: for ', i)
        plus_sentences = temp_df[temp_df[intent]==1]['Split'].to_list()
        minus_sentences = temp_df[temp_df[intent]==-1]['Split'].to_list()

        plus_opp_sent = generate_opposite_sentences(plus_sentences)
        minus_opp_sent = generate_opposite_sentences(minus_sentences)
        overall_opp_sent = plus_opp_sent + minus_opp_sent

        sentiments = [-1]*len(plus_opp_sent)+[1]*len(minus_opp_sent)

        oppdf = pd.DataFrame({'UID': [f'Opp1_{id+k}' for k in range(len(overall_opp_sent))], 'Split': overall_opp_sent, f'{intent}':sentiments})
        id+=len(overall_opp_sent)
        fin_df=pd.concat([temp_df, oppdf])
        fin_df.drop_duplicates(subset='Split', inplace=True)
        result[i]=fin_df
        # continue

    else:

        #Case 2: no sentence present in data for given regex hence nothing much can be done at this point
        if val_count_for_plus_one==0 and val_count_for_minus_one==0:
            print('No data for ', i)
            print(temp_df[intent].value_counts())
            continue

        #Case 3: if both of them are less than min sentence per regex required
        if max([val_count_for_plus_one, val_count_for_minus_one]) < min_sentence_per_regex:
            print('Case 3 for ', i)
            sentences = temp_df['Split'].to_list()

            pos_gen_sent = generate_similar_sentences(sentences, intent, min_sentence_per_regex - val_count_for_plus_one, 'positive')
            neg_gen_sent = generate_similar_sentences(sentences, intent, min_sentence_per_regex - val_count_for_minus_one, 'negative')

            overall_gen_sent = pos_gen_sent + neg_gen_sent
            sentiments = [1]*len(pos_gen_sent)+[-1]*len(neg_gen_sent)
            resdf = pd.DataFrame({'UID': [f'Gen3_{id+k}' for k in range(len(overall_gen_sent))], 'Split': overall_gen_sent, f'{intent}':sentiments})

            fin_df=pd.concat([temp_df, resdf])
            fin_df.drop_duplicates(subset='Split', inplace=True)

            id+=len(overall_gen_sent)
            result[i]=fin_df
            continue

        #Case 4: either of them is greater min requirement hence we need to generate len(+1) sentences for -1 and vice versa for len(-1) sentences for +1
        elif val_count_for_plus_one>val_count_for_minus_one:

            #Ref sentenes
            if not val_count_for_plus_one<=2:
                    plus_ref_sentences = temp_df[temp_df[intent]==1]['Split'].to_list()[min(val_count_for_plus_one, 10)]
            else:
                minus_sentences = temp_df[temp_df[intent]==-1]['Split'].to_list()[min(val_count_for_minus_one, 10)]
                plus_ref_sentences = generate_opposite_sentences(minus_sentences)

            if not val_count_for_minus_one<=2:
                minus_ref_sentences = temp_df[temp_df[intent]==-1]['Split'].to_list()
                minus_ref_sentences = minus_ref_sentences[:min(val_count_for_minus_one, 10)]
            else:
                plus_sentences = temp_df[temp_df[intent]==1]['Split'].to_list()[min(val_count_for_plus_one, 10)]
                minus_ref_sentences = generate_opposite_sentences(plus_sentences)

            #Case 4.1: If both are less than 50, in that case we can just use generation function directly to generate required sentences
            if max(val_count_for_plus_one, val_count_for_minus_one) <= 50:
                plus_gen_sent = generate_similar_sentences(plus_ref_sentences, intent, val_count_for_minus_one, 'positive')
                minus_gen_sent = generate_similar_sentences(minus_ref_sentences, intent, val_count_for_plus_one, 'negative')

            #Case 4.2 either of them is greater than 50 and our upper cap for generation is 50 so we paraphrase some sentences
            elif max(val_count_for_plus_one, val_count_for_minus_one) > 50:
                if val_count_for_minus_one<=50:
                    plus_gen_sent = generate_similar_sentences(plus_ref_sentences, intent, val_count_for_minus_one, 'positive')
                else:

                    plus_gen = generate_similar_sentences(plus_ref_sentences, intent, 50, 'positive')

                    reference_sentences = plus_ref_sentences[:min(val_count_for_minus_one-50, len(plus_ref_sentences))]

                    rem_plus = generate_paraphrased_sentences(reference_sentences, val_count_for_minus_one-50, 1, 'positive')
                    plus_gen_sent = plus_gen + rem_plus

                if val_count_for_plus_one<=50:
                    minus_gen_sent = generate_similar_sentences(minus_ref_sentences, intent, val_count_for_plus_one, 'negative')
                else:
                    minus_gen = generate_similar_sentences(minus_ref_sentences, intent, 50, 'negative')

                    reference_sentences = minus_ref_sentences[:min(val_count_for_plus_one-50, len(minus_ref_sentences))]

                    rem_minus = generate_paraphrased_sentences(reference_sentences, val_count_for_plus_one-50, 1, 'negative')
                    minus_gen_sent = minus_gen + rem_minus

            overall_gen_sent = plus_gen_sent + minus_gen_sent
            sentiments = [1]*len(plus_gen_sent)+[-1]*len(minus_gen_sent)
            resdf = pd.DataFrame({'UID': [f'Gen4_{id+k}' for k in range(len(overall_gen_sent))], 'Split': overall_gen_sent, f'{intent}':sentiments})


            fin_df=pd.concat([temp_df, resdf])
            fin_df.drop_duplicates(subset='Split', inplace=True)

            id+=len(overall_gen_sent)
            result[i]=fin_df

            # continue
    print(fin_df[intent].value_counts())

Ongoing generation:   0%|          | 0/7 [00:00<?, ?it/s]Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Ongoing wide[- ]{0,3}angle
Camera
1.0    4
Name: count, dtype: int64
Case 3 for  wide[- ]{0,3}angle


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.
Ongoing generation:  14%|█▍        | 1/7 [03:29<20:54, 209.10s/it]Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.


Ongoing \bvide?o?s?\b(?!.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b).*\bm?i?d?nightt?i?m?e?\b(?!.*day|.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b)|\bm?i?d?nightt?i?m?e?\b(?!.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b).*\bvide?o?s?\b(?!.*day|.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b)
Camera
 1.0    2
-1.0    1
Name: count, dtype: int64
Case 3 for  \bvide?o?s?\b(?!.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b).*\bm?i?d?nightt?i?m?e?\b(?!.*day|.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b)|\bm?i?d?nightt?i?m?e?\b(?!.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b).*\bvide?o?s?\b(?!.*day|.*\bimages?\b|.*\bpictures?\b|.*\bpics?\b|.*\bportraits?\b|.*\bphotos?\b)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.
Ongoing generation:  29%|██▊       | 2/7 [08:13<21:05, 253.19s/it]Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.


...

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.
Ongoing generation:  71%|███████▏  | 5/7 [15:53<05:59, 179.59s/it]Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.


Ongoing mega \bpixel
Camera
1.0    1
Name: count, dtype: int64
Case 3 for  mega \bpixel


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.
Ongoing generation:  86%|████████▌ | 6/7 [18:38<02:54, 174.57s/it]Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.


Ongoing (?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b).*\bvide?o?s?\b.*\bm?i?d?nightt?i?m?e?\b|\bvide?o?s?\b.*(?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b).*\bm?i?d?nightt?i?m?e?\b|\bm?i?d?nightt?i?m?e?\b.*(?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b).*\bvide?o?s?\b|\bm?i?d?nightt?i?m?e?\b.*\bvide?o?s?\b.*(?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b)
Camera
1.0    2
Name: count, dtype: int64
Case 3 for  (?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b).*\bvide?o?s?\b.*\bm?i?d?nightt?i?m?e?\b|\bvide?o?s?\b.*(?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b).*\bm?i?d?nightt?i?m?e?\b|\bm?i?d?nightt?i?m?e?\b.*(?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b).*\bvide?o?s?\b|\bm?i?d?nightt?i?m?e?\b.*\bvide?o?s?\b.*(?:\bimages?\b|\bpictures?\b|\bpics?\b|\bportraits?\b|\bphotos?\b)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesconfig'>.
Ongoing generation: 100%|██████████| 7/7 [20:51<00:00, 178.77s/it]


In [ ]:
with open('/content/drive/Shareddrives/sentence_genration/misc/Camera_generated_29_Jul_Part7.pkl', 'wb') as f:
  pickle.dump(result, f)